In [6]:
import cv2
from ultralytics import YOLO
import cvzone
import numpy as np
import pandas as pd
from collections import Counter  # Import Counter from collections module
import glob

In [7]:
model = YOLO('./runs/detect/train/weights/best.pt')

In [8]:
# my_file = open("coco.txt", "r")
# data = my_file.read()
# class_list = data.split("\n")

def object(img):
    results = model.predict(img)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")
    object_classes = []

    for index, row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        conf = float(row[4])# Get confidence score from column 4
        d=int(row[5])
#         c=class_list[d]
        obj_class = "orange" #class_list[d]
        
        # Show confidence score with two decimal places
        confidence_text = f"{obj_class}: {conf:.2f}"
        
        
        # Adjust bounding box and text parameters based on the image and your preferences
        bounding_box_color = (0, 0, 255)  # Red color for the first bounding box
        text_color = (255, 255, 255)  # White color for text
        text_font = cv2.FONT_HERSHEY_SIMPLEX
        text_size = 1
        text_thickness = 2
        text_offset_x = 5  # Adjust horizontal offset for text within box
        text_offset_y = 5  # Adjust vertical offset for text within box

        # Draw bounding box with adjusted color
        cv2.rectangle(img, (x1, y1), (x2, y2), bounding_box_color, 2)

        # Calculate text dimensions for background rectangle
        (text_width, text_height) = cv2.getTextSize(confidence_text, text_font, text_size, text_thickness)[0]
        text_background_width = text_width + 2 * text_offset_x  # Add some padding
        text_background_height = text_height + 2 * text_offset_y

        # Draw background rectangle with adjusted color
        cv2.rectangle(img, (x1, y1 - text_background_height), (x1 + text_background_width, y1), bounding_box_color, -1)  # Filled rectangle

        # Display class and confidence score within the box using adjusted parameters
        text_x = x1 + text_offset_x
        text_y = y1 - text_offset_y  # Adjust y-coordinate for text placement within box
        cv2.putText(img, confidence_text, (text_x, text_y), text_font, text_size, text_color, text_thickness)
        object_classes.append(obj_class)
        
#         # Change bounding box color to red
#         cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)        
#         # Display class and confidence score as text within the box
#         cv2.putText(img, confidence_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA) # White text
#         #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
#         #cvzone.putTextRect(img, f'{obj_class}', (x2, y2), 1, 1)

    return object_classes






In [9]:
def count_objects_in_image(object_classes):
    counter = Counter(object_classes)
    print("Object Count in Image:")
    for obj, count in counter.items():
        print(f"{obj}s: {count}")

In [10]:
path = "./Test Images/1.jpg"
for file in glob.glob(path):
    img = cv2.imread(file)
    img = cv2.resize(img, (640, 640))
    object_classes = object(img)
    count_objects_in_image(object_classes)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


0: 640x640 31 oranges, 178.5ms
Speed: 14.4ms preprocess, 178.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Object Count in Image:
oranges: 31
